How to mock and pass through?



In [1]:
var execSync = require('child_process').execSync;
try {
    require.resolve('automock');
    require.resolve('bluebird');
} catch (e) {
    execSync('npm install automock bluebird');
}

var Promise = require('bluebird');
var automock = require('automock');
var promisifyMock = (req, dep) => {
    let ctx;
    ctx = automock.mockValue(req, {
        stubCreator: (name) => {
            var orig = Promise.promisify(req[name.split('.')[1]], {
                multiArgs: true,
                context: req
            });
            //console.log('create stub ' + name);
            return function () {
                console.log(name + ' (' + arguments[0] + ') in ' + JSON.stringify(dep));
                return orig.apply(null, arguments);
            };
        }
    });
    return ctx;
};
promisifyMock;


[Function: promisifyMock]

Create an RPC socker.io server for any module using promisify?



In [2]:
var execSync = require('child_process').execSync;
try {
    require.resolve('automock');
    require.resolve('socket.io');
} catch (e) {
    execSync('npm install automock socket.io');
}
var automock = require('automock');

var deps = {};
var sockifyRequire = function (dep) {
    if(typeof deps[dep] != 'undefined') {
        return deps[dep];
    }
    console.log('requiring ' + dep);
    // TODO: add functionality to replace stubCreator with client side code, e.g. (name) => io.to('LogService').emit('handle', args)
    var req = require(dep);
    let ctx;
    ctx = automock.mockValue(req, {
        stubCreator: (name) => {
            var orig = req[name.split('.')[1]];
            return function () {
                console.log(name + ' ( ' + arguments[0] + ' ) ');
                return orig.apply(null, arguments);
            }
        }
    });
    deps[dep] = ctx;
    return ctx;
}

var sockifyServer = (port) => {
    var io = require('socket.io').listen(port);
    var socketlist = [];
    io.sockets.on('connection', function (socket) {
        socketlist.push(socket);
        socket.on('handler', (dep, code, fn) => {
            // TODO: add a utility function to include specific code as the handler for automock as a replacement for requiring the actual dependency
            // TODO: register the handler for the dependency
        });
        
        socket.on('require', function (dep, fn) {
            sockifyRequire(dep);
            fn('done');
        });
        
        socket.on('call', function (dep, name) {
            var args = [];
            for(var i = 2; i < arguments.length; i++) {
                args[args.length] = arguments[i];
            }
            deps[dep][name].apply(null, args);
        });
        
        // TODO: organize utility functions
        socket.on('close', function () {
            socketlist.splice(socketlist.indexOf(socket), 1);
            socketlist.forEach(function(socket) {
                socket.destroy();
            });
            io.server.close();
        });
    });
    return io;
};
sockifyServer;


// TODO: output interactive angular component for controlling this server



[Function: sockifyServer]

In [3]:
var execSync = require('child_process').execSync;
try {
    require.resolve('socket.io-client');
} catch (e) {
    execSync('npm install socket.io-client');
}

var client = require('socket.io-client');
var sockifyClient = (req, dep, host) => {
    let ctx;
    ctx = automock.mockValue(req, {
        stubCreator: (name) => {
            return function () {
                var args = ['call', dep, name.split('.')[1]];
                for(var i = 0; i < arguments.length; i++) {
                    args[args.length] = arguments[i];
                }
                socket.emit.apply(socket, args);
            };
        }
    });
    var promises = promisifyMock(ctx, dep);
    promises.___close = () => socket.emit('close');
    var socket = client.connect(host);
    socket.on('connect', function () {
        // TODO: socket.emit('handler') service provider
        socket.emit('require', dep, function () {
            
        });
        socket.on('resolve', function () {
            
        });
    });
    return promises;
};
sockifyClient;



[Function: sockifyClient]

Check if sockify server is running?

In [ ]:
var tryReset = false;
try {
    var io = sockifyServer(8098);
} catch (e) {
    if(e.code !== 'EADDRINUSE') {
        throw e;
    } else {
        tryReset = true;
    }
}
var fs = sockifyClient(require('fs'), 'fs', 'http://localhost:8098');
if(tryReset) {
    fs.___close();
    var io = sockifyServer(8098);
}


In [4]:

$$.async();
fs.unlink('test.file')
    .catch(e => console.log(e))
    .then(() => fs.writeFile('test.file', 'some output'))
    .then(() => fs.readFile('test.file'))
    .then(r => r.toString())
    .then(r => $$.sendResult(r))
    .then(() => fs.___close())
    .catch(e => $$.sendError(e));



(none).unlink (test.file) in "fs"


(node:39509) DeprecationWarning: sprintf() will be removed in the next major release, use the sprintf-js package instead.


requiring fs
(none).unlink ( test.file ) 
(none).writeFile (test.file) in "fs"
(none).writeFile ( test.file ) 
(none).readFile (test.file) in "fs"
(none).readFile ( test.file ) 


'some output'